In [1]:
import pandas as pd

# load data
weapons = pd.read_csv('mhw_weapon_properties.csv', index_col='Name')
weapon_types = pd.read_csv('mhw_weapon_type_properties.csv', index_col='Weapon Type')
weapon_sharpness = pd.read_csv('mhw_sharpness_damage_multipliers.csv', index_col='Damage Type')
monsters = pd.read_csv('mhw_monster_damage_multipliers.csv', index_col='Monster')

# Weapon type motion values are mean from all values at https://www.reddit.com/r/MonsterHunter/comments/7v0pp3/mhworld_motion_values_compiled/
# Weapon type average hit per second are from https://www.reddit.com/r/MonsterHunter/comments/4e9o13/weapons_by_attack_speed/
# Monster damage modifiers are average values from https://honeyhunterworld.com/mhwb/
# Weapon properties are from https://monsterhunterworld.wiki.fextralife.com/Weapons


In [8]:
import itertools as it

# Define class for DPS calculation

class dps_calculator:
    
    def __init__(self):
        self.weapons = ["Daora's Entom", 'Tyrannis Glaive II']
        self.monsters = ['Diablos', 'Black Diablos', 'Teostra', "Xeno'jiiva", 'Kirin']
        self.buffs = {'affinity_modifier'  : 0,
                      'attack_boost'       : 22,
                      'attack_modifier'    : 0,
                      'elemental_boost'    : 0,
                      'elemental_modifier' : 0,
                      'critical_element'   : 0,
                      'handicraft'         : 1,
                      'note'               : 'Default'}
        self.default_buffs = {'affinity_modifier'  : 0,
                              'attack_boost'       : 22,
                              'attack_modifier'    : 0,
                              'elemental_boost'    : 0,
                              'elemental_modifier' : 0,
                              'critical_element'   : 0,
                              'handicraft'         : 1,
                              'note'               : 'Default'}

    # set armor and buff modifiers
    def set_modifiers(self, **kwargs):    
        for buff in self.buffs:
            if buff in kwargs:
                self.buffs[buff] = kwargs[buff]
            else:
                self.buffs[buff] = self.default_buffs[buff]

    # calculate DPS for weapons and monsters
    def calculate_dps(self, data):
        for m, w in it.product(self.monsters, self.weapons):
            t = weapons.loc[w, 'Type']
            if self.buffs['handicraft'] == 1:
                s = weapons.loc[w, 'Handicraft Sharpness']
            else:
                s = weapons.loc[w, 'Sharpness']    
            raw_dps = ((weapons.loc[w, 'Damage'] / weapon_types.loc[t, 'Multiplier']) + self.buffs['attack_boost']) * \
                      ((self.buffs['attack_modifier'] / 100) + 1) * \
                      weapon_sharpness.loc['Raw', s] * \
                      (((weapons.loc[w, 'Affinity'] + self.buffs['affinity_modifier']) / 400) + 1) * \
                      (weapon_types.loc[t, 'Average Motion Multiplier'] / 100) * \
                      (monsters.loc[m, weapons.loc[w, 'Raw Type']] / 100) * \
                      weapon_types.loc[t, 'Average Hits Per Second']
            elem_dps = ((weapons.loc[w, 'Elemental Damage'] + self.buffs  ['elemental_boost']) / 10) * \
                       ((self.buffs['elemental_modifier'] / 100) + 1) * \
                       weapon_sharpness.loc['Elemental', s] * \
                       ((((weapons.loc[w, 'Affinity'] + self.buffs['affinity_modifier']) / 400) * self.buffs['critical_element']) + 1) * \
                       (monsters.loc[m, weapons.loc[w, 'Elemental Type']] / 100) * \
                       weapon_types.loc[t, 'Average Hits Per Second']
            data.append([w, self.buffs['note'], m, raw_dps, elem_dps, raw_dps + elem_dps])
            

In [9]:
# initialize
dps_data = []
dps_calc = dps_calculator()

# calculate with different buffs
dps_calc.set_modifiers()
dps_calc.calculate_dps(dps_data)
dps_calc.set_modifiers(affinity_modifier = 50, note = 'With Affinity Booster')
dps_calc.calculate_dps(dps_data)
dmg_df = pd.DataFrame(dps_data, columns=['Weapon', 'Note', 'Against', 'Raw DPS', 'Elem DPS', 'Total DPS'])

# display
dmg_df.sort_values(by = ['Against', 'Total DPS'])

,Weapon,Note,Against,Raw DPS,Elem DPS,Total DPS
3,Tyrannis Glaive II,Default,Black Diablos,46.558991,0.000000,46.558991
2,Daora's Entom,Default,Black Diablos,39.158115,7.857000,47.015115
12,Daora's Entom,With Affinity Booster,Black Diablos,43.933495,7.857000,51.790495
13,Tyrannis Glaive II,With Affinity Booster,Black Diablos,52.850746,0.000000,52.850746
1,Tyrannis Glaive II,Default,Diablos,49.075693,0.000000,49.075693
0,Daora's Entom,Default,Diablos,41.274770,7.857000,49.131770
10,Daora's Entom,With Affinity Booster,Diablos,46.308278,7.857000,54.165278
11,Tyrannis Glaive II,With Affinity Booster,Diablos,55.707543,0.000000,55.707543
8,Daora's Entom,Default,Kirin,32.808150,4.583250,37.391400
9,Tyrannis Glaive II,Default,Kirin,39.008884,0.000000,39.008884
